In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from geopy.distance import distance
import swifter

In [ ]:
import sqlalchemy
import pymysql

In [ ]:
class CloudSQLDAO:
    def __init__(self, auth):

        if(auth):
            db_user = 'root'
            db_pass = 'thanks123'
            db_name = 'drivesafe_app_accidents'
            db = sqlalchemy.create_engine('mysql+pymysql://{}:{}@127.0.0.1/{}'.format(db_user,db_pass,db_name))
            self.conn = db.connect()
        else:
            pass
            '''
            with open("config/cfg.yml", 'r') as ymlfile:
                cfg = yaml.load(ymlfile)
                self.db_user=cfg['db_user']
                self.db_pass=cfg['db_pass']
                self.db_name = cfg['db_name']
                self.cloud_sql_instance_name = cfg['cloud_sql_instance_name']

            self.db = sqlalchemy.create_engine(
                # Equivalent URL:
                # mysql+pymysql://<db_user>:<db_pass>@/<db_name>?unix_socket=/cloudsql/<cloud_sql_instance_name>
                sqlalchemy.engine.url.URL(
                    drivername="mysql+pymysql",
                    username=self.db_user,
                    password=self.db_pass,
                    database=self.db_name,
                    query={"unix_socket": "/cloudsql/{}".format(self.cloud_sql_instance_name)},
                )
            )
            '''
    def get_accidents_by_zipcode(self, zipcode):
        sql = "select ID,Start_Lat,Start_Lng from accidents where Zipcode={}".format(zipcode)
        return pd.read_sql(sql, self.conn)
    

In [ ]:
dao = CloudSQLDAO(auth=True)

In [ ]:
dao.get_accidents_by_zipcode(94401)

In [ ]:
o_df = pd.read_csv('US_Accidents_Dec19.csv')

In [3]:
import json
output_map = {}
output_map['id'] = 1
output_map['warning_score'] = 2
output_map['warning_level'] = 1 #can be between 1-3
print(json.dumps(output_map))

{"id": 1, "warning_score": 2, "warning_level": 1}


In [ ]:
df.head()

In [ ]:
df.Traffic_Signal.value_counts()

In [ ]:
distance

In [ ]:
tempe = df[df.Zipcode == '85281']

In [ ]:
tempe.head()

In [ ]:
dfla = df[df.City == 'Los Angeles']

In [ ]:
dfla.shape

In [ ]:
dfla.Zipcode

In [ ]:
def filter_coords(start, dest, latlong, thresh):
    '''
    Filter set of GPS locations from a given region based on (start, dest)  using triangle method.
    
    args:
        start: GPS start lat-long (tuple).
        dest: GPS end lat-long (tuple).
        latlong: Zip of all lat-long tuples for a given region.
        thresh: Max thresh for sum of distances for GPS loc to be filtered.
        
    returns:
        filtered_search_list: filtered list of GPS locations.
        mask: Indicator for each entry in a given region; 1-> In search lise, 0-> otherwise.
    '''
    filtered_search_list = []
    mask = []
    for coord in latlong:
        if (distance(start, coord).miles + distance(dest, coord).miles) <= thresh:
            filtered_search_list.append(coord)
            mask.append(1)
        else:
            mask.append(0)
    return filtered_search_list, mask

In [ ]:
def check_accident_zone(current_loc, filtered_search_list, thresh=0.5):
    '''
    Check if a given location is in accident prone zone
    
    args:
        curent_loc: Query / current GPS location of vehicle.
        filtered_Search_list: Optimized search list of GPS locations based on (start, dest), calc by triangle method.
        thresh: Minimum distance from danger zone for warning to be considered.
    
    returns:
        True -> In danger zone.
        False -> In safe zone.
    '''
    ctr = 0
    danger_signal = 0 ## To be modified later according to what we determine is a danger zone (number of accidents in a given region)
    for loc in filtered_search_list:
        if distance(current_loc, loc).miles <= thresh:
            ctr += 1
    if ctr > danger_signal:
        return True
    return False

In [ ]:
lats = dfla.Start_Lat.astype(float).values
longs = dfla.Start_Lng.astype(float).values

In [ ]:
src = (34.016993, -118.496299)
dest = (33.436832, -111.957603)
distance(src, dest).miles

In [ ]:
%%time
accident_latlong = zip(lats, longs)
start = (34.009842, -118.495851) ## Santa Monica Pier
dest = (33.944360, -118.408647) ## LAX Airport
dist_thresh = distance(start, dest).miles + 5.0 ## Change according to experiments
coords, mask = filter_coords(start, dest, latlong=accident_latlong, thresh=dist_thresh)

In [ ]:
def test_search(row, start, dest, thresh):
    coord = (float(row['Start_Lat']),float(row['Start_Lng']))
    if (distance(start, coord).miles + distance(dest, coord).miles) <= thresh:
        return 1
    return 0
res = dfla.apply(test_search,args=(start,dest,dist_thresh),axis=1)

In [ ]:
%%time


In [ ]:
res.sum()

In [ ]:
%%time
len(coords)

In [ ]:
len(lats)

In [ ]:
dest

In [ ]:
distance(start, dest).miles

In [ ]:
df.shape

In [ ]:
df.City.unique().shape[0]

In [ ]:
df.Street

In [ ]:
df = pd.read_csv('CA_Accidents_Filtered.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df = df.dropna(subset=['Zipcode'] )
df['Zipcode'] = df['Zipcode'].apply(lambda x: str(x).split('-')[0])

In [ ]:
df.to_csv('CA_cleaned.csv',header=False,index=False)

In [ ]:
o_df.Zipcode.isnull().sum()

In [ ]:
df['Zipcode'].unique().shape

In [ ]:
a = 1
b =1

In [ ]:
zipcode_accidents = df[df['Zipcode']=='95237']

In [ ]:
def infer_warning_level(curr_lat,curr_lng,zipcode_accidents):
    def get_points_within_radius(row,curr_lat,curr_lng,thresh=0.2):
        curr_loc =(curr_lat,curr_lng)
        row_loc = (row['Start_Lat'],row['Start_Lng'])
        dist = distance(row_loc,curr_loc).miles
        if(dist<=thresh):
            return True
        else:
            return False
    result = zipcode_accidents.apply(get_points_within_radius,args=(curr_lat,curr_lng),axis=1)
    return result.sum()

In [ ]:
curr_lat,curr_lng = (38.161133, -121.152646)
infer_warning_level(curr_lat,curr_lng,zipcode_accidents)

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import distance
locator = Nominatim(user_agent="myGeocoder")
zipcode_object = locator.reverse((curr_lat,curr_lng))
curr_zipcode = zipcode_object.raw['address']['postcode']

In [ ]:
[2:16 AM, 5/5/2020] Venkatesh Asu: 33.465523734896585 -111.95820843641205
[2:16 AM, 5/5/2020] Venkatesh Asu: 33.465521696022314 -111.95874680390439
[2:16 AM, 5/5/2020] Venkatesh Asu: 33.4655247726229 -111.95928520900634
[2:16 AM, 5/5/2020] Venkatesh Asu: 33.46552784922348 -111.95982361410829
[2:16 AM, 5/5/2020] Venkatesh Asu: 33.465533709054185 -111.96036196203275

In [ ]:
distance(('33.465523734896585','-111.95820843641205'),('33.465521696022314', '-111.95874680390439')).miles

In [ ]:
distance((33.465523734896585','-111.95820843641205'),('33.4655247726229', '-111.95928520900634')).miles

In [ ]:
500m/sec

In [ ]:
zipcode_object = locator.reverse((curr_lat,curr_lng))
curr_zipcode = zipcode_object.raw['address']['postcode']